In [1]:
import numpy as np
import os
import subprocess
import json
import calendar
import pandas as pd
import geopandas as gpd
import shapely
import random
import datetime
from shapely.geometry import Point
from geopandas import GeoDataFrame
import warnings
from scipy.spatial.distance import cdist

In [69]:
# testing
result = subprocess.run(["curl", 'http://localhost:5000/route/v1/driving/13.388860,52.517037;13.397634,52.529407?overview=false'], stdout=subprocess.PIPE)
resultStr = result.stdout.decode("utf-8")
print(resultStr)
resultDict = json.loads(resultStr)
distance = resultDict['routes'][0]['distance']
print(distance)

result = subprocess.run(["curl", 'http://router.project-osrm.org/route/v1/driving/13.388860,52.517037;13.397634,52.529407?overview=false'], stdout=subprocess.PIPE)
resultStr = result.stdout.decode("utf-8")
print(resultStr)
resultDict = json.loads(resultStr)
distance = resultDict['routes'][0]['distance']
print(distance)


{"waypoints":[{"location":[-86.83958,45.395992],"name":"Old Camp Road","hint":"vIoBgIWSAYA-AAAAAAAAAOMDAAAAAAAAPgAAAAAAAADjAwAAAAAAAEkBAIDk7tL6GLC0AjxMzACtWCEDCgB_BXInndc="},{"location":[-86.83958,45.395992],"name":"Old Camp Road","hint":"vIoBgIWSAYA-AAAAAAAAAOMDAAAAAAAAPgAAAAAAAADjAwAAAAAAAEkBAIDk7tL6GLC0AoJuzAD_iCEDCgB_BXInndc="}],"routes":[{"distance":0,"duration":0,"weight":0,"weight_name":"routability","legs":[{"distance":0,"duration":0,"weight":0,"summary":"","steps":[]}]}],"code":"Ok"}
0
{"routes":[{"legs":[{"summary":"","weight":435.4,"duration":370.2,"steps":[],"distance":1879.4}],"weight_name":"routability","weight":435.4,"duration":370.2,"distance":1879.4}],"waypoints":[{"hint":"x_cJgCfqRoYbAAAAGwAAAAYAAAAAAAAAGwAAABsAAAAGAAAAAAAAAKPZAAAATMwAqVghAzxMzACtWCEDAQDfCmnIRbQ=","name":"Friedrichstraße","location":[13.3888,52.517033]},{"hint":"XBsUgJZ77YEKAAAACAAAABsBAAAAAAAACgAAAAgAAAAbAQAAAAAAAKPZAAB_bswAGIkhA4JuzAD_iCEDAgCfEGnIRbQ=","name":"Torstraße","location":[13.397631,52.529

In [72]:
# testing
dist = distance_street(13.388860,52.517037,13.397634,52.529407)
print(dist)
dist = distance_street(43.090407,-87.929521,43.095108,-87.947162)
print(dist)
dist = distance_street(-87.929521,43.090407,-87.947162,43.095108)
print(dist)
dist = distance_street(43.090407754371284,-87.92952162479058,43.09510847013696,-87.94716283236194)
print(dist)

http://localhost:5000/route/v1/driving/52.517037,13.38886;52.529407,13.397634?overview=false
{'waypoints': [{'location': [-87.804176, 42.490471], 'name': '', 'hint': 'z2YOgPD9EICFAQAAAAAAAAAAAAAAAAAAwwAAAAAAAAAAAAAAAAAAAIsEAIDwNsT6Z1qIAq1YIQM8TMwAAADPCnInndc='}, {'location': [-87.804176, 42.490471], 'name': '', 'hint': 'z2YOgPD9EICFAQAAAAAAAAAAAAAAAAAAwwAAAAAAAAAAAAAAAAAAAIsEAIDwNsT6Z1qIAv-IIQOCbswAAADPCnInndc='}], 'routes': [{'distance': 0, 'duration': 0, 'weight': 0, 'weight_name': 'routability', 'legs': [{'distance': 0, 'duration': 0, 'weight': 0, 'summary': '', 'steps': []}]}], 'code': 'Ok'}
0
http://localhost:5000/route/v1/driving/-87.929521,43.090407;-87.947162,43.095108?overview=false
{'waypoints': [{'location': [-87.929531, 43.090407], 'name': 'North 15th Street', 'hint': '-2gDgJXAE4CQAAAAPwAAAAAAAAAAAAAAkAAAAD8AAAAAAAAAAAAAABkBAABFTcL654GRAk9NwvrngZECAABfC3Inndc='}, {'location': [-87.947163, 43.095168], 'name': 'West Roosevelt Drive', 'hint': 'V9sCgOycE4B0AAAAJAAAAAAAAAAAAAAAd

In [71]:
# just for debugging
def distance_street(lat_1, lon_1, lat_2, lon_2):

    query = 'http://localhost:5000/route/v1/driving/%s,%s;%s,%s?overview=false' % (str(lon_1), str(lat_1), str(lon_2), str(lat_2))
    print(query)
    result = subprocess.run(["curl", query], stdout=subprocess.PIPE)
    resultStr = result.stdout.decode("utf-8")
    resultDict = json.loads(resultStr)
    print(resultDict)
    streetDistance = resultDict['routes'][0]['distance']
    #print(distance)

    return streetDistance

In [102]:
class KMeans(object):
    def __init__(self, k=8, distance = 'euclid', max_iter=100):
        self.k = k
        if (distance == 'euclid'):
            self._distance = 'euclidean'
        elif (distance == 'haversine'):
            self._distance = self._distance_haversine
        elif (distance == 'manhattan'):
            self._distance = self._distance_manhattan
        elif (distance == 'street'):
            self._distance = self._distance_street
        else:
            raise Exception('Invalid distance metric')
    
    def _step(self):
        """Compute distance, assign groups, recompute centers"""
        distance = cdist(self.X,self.cluster_centers,metric=self._distance)
        self.labels = distance.argmin(1)
       # centers = np.zeros((self.k,2))
        for cluster in range(self.k):
            points = self.X[self.labels == cluster]
            if len(points) == 0:
                distance = cdist(self.X,np.delete(self.cluster_centers,cluster,0),metric=self._distance)
                mean_dist = np.mean(distance,0)
                self.cluster_centers[cluster] = mean_dist.argmax()
            else:
                self.cluster_centers[cluster] = np.mean(points,0)
       # self.cluster_centers = centers
        
    def _distance_haversine(self,a,b):
        lat_1, lon_1, lat_2, lon_2 = map(np.radians,[a[0],a[1],b[0],b[1]])
        d_lat = lat_2 - lat_1
        d_lon = lon_2 - lon_1
        
        arc = np.sin(d_lat/2.0)**2 + np.cos(lat_1)*np.cos(lat_2)*np.sin(d_lon/2)**2
        
        c = 2 * np.arcsin(np.sqrt(arc))
        km = 6372.8 * c
        return km
    
    def _distance_manhattan(self, a, b):
        lat_1 = a[0] 
        lon_1 = a[1]
        lat_2 = b[0]
        lon_2 = b[1]
        manhattanDistance = abs(lat_2 - lat_1) + abs(lon_2 - lon_1)
        return manhattanDistance
    
    def _distance_street(self, a, b):
        lat_1 = a[0] 
        lon_1 = a[1]
        lat_2 = b[0]
        lon_2 = b[1]

        query = 'http://localhost:5000/route/v1/driving/%s,%s;%s,%s?overview=false' % (str(lon_1), str(lat_1), str(lon_2), str(lat_2))
        #print(query)
        result = subprocess.run(["curl", query], stdout=subprocess.PIPE)
        resultStr = result.stdout.decode("utf-8")
        #print(resultStr)
        resultDict = json.loads(resultStr)
        streetDistance = float(resultDict['routes'][0]['distance'])
        #print(str(lat_1) + " " + str(lat_2) + ": " + str(streetDistance))
        
        return streetDistance
   
    
    def _init_centers(self, X):
        unique = np.unique(X, axis=0)
        index = np.random.permutation(len(unique))[:self.k]
        return unique[index]
    
    def fit(self,X, centers = None):
        '''Expects centers to be inputted, if not random'''
        self.labels = np.zeros(len(X))
        self.X = X
        if centers is not None:
            self.cluster_centers = centers 
        else:
            self.cluster_centers = self._init_centers(X)
        old_centers = np.zeros((self.k,2))
    #    self.i = 0
        while(not np.array_equal(old_centers, self.cluster_centers)):
            old_centers = self.cluster_centers.copy()
            self._step()
         #   self.i+=1

In [101]:
demographics = gpd.read_file('./census.geoJSON')

def gen_coords(loc):
    data = loc[1:-1].split(',')
    data = list((np.float(data[0]), np.float(data[1])))
    x.append(data[1])
    y.append(data[0])
    return [data[0],data[1]]

def point_similarity(X,geo_labels, euc_labels,k):
    '''For an inputted series of points, geodesic labels, euclidean labels, and k-value
       returns the point-similarity index per geodesic cluster
    '''

    euc_cluster_totals = np.zeros(k,dtype=np.int)
    geo_euc_composition = [np.zeros(k,dtype=np.int)* 1 for i in range(k)]
    
    for index,point in enumerate(geo_labels):
        euc_cluster_totals[euc_labels[index]] += 1
        geo_euc_composition[point][euc_labels[index]] += 1
    
    point_sim = []
    for geo_cluster in range(k):
        sim = 0
        for euc_cluster in range(k):
            matching_points = geo_euc_composition[geo_cluster][euc_cluster]
            euc_percentage = matching_points / euc_cluster_totals[euc_cluster]
            geo_percentage = matching_points / np.sum(geo_euc_composition[geo_cluster])
            sim += euc_percentage * geo_percentage
        point_sim.append(sim)

    return np.array(point_sim)

def minority_probability(X,cluster_number,geo_labels,demographics):
        points = X[geo_labels == cluster_number]
        # geoJSON puts points in Long/Lat order
        # but points are in lat/long earlier
        hull = shapely.geometry.multipoint.MultiPoint([[p[1],p[0]] for p in points]).convex_hull
  
        pop = np.zeros(7)
        for index in range(len(demographics)):
            census_tract = demographics.loc[index,'geometry']
            intersect = hull.intersection(census_tract)
            overlap = intersect.area/census_tract.area
            if (overlap != 0):
                pop = pop + (np.array(demographics.loc[index,['White','Black or African American', 'American Indian and Ala Native',
                   'Asian','Native Hawaiian/other Pac Isl', 'Multiple Race',
                   'Other Race']]) * overlap)
        
        if (np.all(pop ==0)):
            return 0
        
        return (pop[1:]/np.sum(pop)).sum()

def bias_index(X, geo_labels, euc_labels, demographics, k):
    if np.all(geo_labels == euc_labels):
        return 0

    dissimilarity_index = 1 - point_similarity(X,geo_labels,euc_labels,k)
    minority_prob = np.array([minority_probability(X,cluster,geo_labels,demographics) 
                              for cluster in range(k)])
    
    potential_bias = minority_prob * dissimilarity_index
    return potential_bias.mean()


In [3]:
columns = ['year', 'k']
for crime in ['theft', 'motor', 'assault', 'robbery']:
    for month in calendar.month_abbr:
        if month != "":
            columns.append(crime + "_" + month.lower())

print(len(columns))

frame_list = []
for year in range(2005,2017):
    for k in range(2,11):
        year_list = [str(year)]
        year_list.append(k)
        year_list.extend([np.nan for i in range(48)])
        #for _ in range(48):
        #    year_list.append(np.nan) 
        frame_list.append(year_list)
    
bias_frame = pd.DataFrame(data=frame_list, columns=columns)

bias_frame.head(12)

50


,year,k,theft_jan,theft_feb,theft_mar,theft_apr,theft_may,theft_jun,theft_jul,theft_aug,...,robbery_mar,robbery_apr,robbery_may,robbery_jun,robbery_jul,robbery_aug,robbery_sep,robbery_oct,robbery_nov,robbery_dec
0,2005,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2005,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2005,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2005,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2005,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2005,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2005,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2005,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2006,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
def store_bias(year, month, crime_type, bias_value, k):
    year_array = np.array(bias_frame.year == year) 
    k_array = np.array(bias_frame.k == k)
    index = np.logical_and(year_array, k_array)
    biasFrameColName = crime_type + "_" + month
    bias_frame.loc[index, biasFrameColName] = bias_value

In [96]:
def gen_coords(loc):
    data = loc[1:-1].split(',')
    data = list((np.float(data[0]), np.float(data[1])))
    x.append(data[1])
    y.append(data[0])
    return [data[0],data[1]]

def percent_similarity(a,b):
    return len(a[a==b])/len(a)

def KMeansInstance(year, X, krange, file, df):
    for k in krange:
        start = datetime.datetime.now()
        print(folder + " " + file + " " + str(k))
        euclid = KMeans(k = k, distance = 'euclid')
        #print("Done Euclid")
        geodesic = KMeans(k = k, distance = 'haversine')
        #print("Done Geodesic")
        manhattan = KMeans(k = k, distance = 'manhattan')
        #print("Done Manhattan")
        street = KMeans(k = k, distance = 'street')
        print("Done Street")

        centers = geodesic._init_centers(X)

        euclid.fit(X, centers = centers)
        #print('Fitted Euclid')
        geodesic.fit(X, centers = centers) 
        #print('Fitted Geodesic')
        manhattan.fit(X, centers = centers)
        #print('Fitted Manhattan')
        street.fit(X, centers = centers)
        print('Fitted Street')

        print('Calculating Bias')
        bias_GeodesicEuclid = bias_index(X, geodesic.labels, euclid.labels, demographics, k)

        bias_ManhattanGeodesic = bias_index(X, manhattan.labels, geodesic.labels, demographics, k)
        bias_ManhattanEuclid = bias_index(X, manhattan.labels, euclid.labels, demographics, k)

        bias_StreetEuclid = bias_index(X, street.labels, euclid.labels, demographics, k)
        bias_StreetGeodesic = bias_index(X, street.labels, geodesic.labels, demographics, k)
        bias_StreetManhattan = bias_index(X, street.labels, manhattan.labels, demographics, k)

        print('Storing Bias')
        #  print(folder,file,bias_val,k)
        month = file.split('.csv')[0].split("_")[-1]
        crime_type = file.split(month)[0]
        store_bias(year, month, crime_type, bias_GeodesicEuclid, k)
        store_bias(year, month, crime_type, bias_ManhattanGeodesic, k)
        store_bias(year, month, crime_type, bias_ManhattanEuclid, k)
        store_bias(year, month, crime_type, bias_StreetEuclid, k)
        store_bias(year, month, crime_type, bias_StreetGeodesic, k)
        store_bias(year, month, crime_type, bias_StreetManhattan, k)

        ("Saving")
        df.loc[:,'e_cluster' + 'K' + str(k)] = euclid.labels.copy()
        df.loc[:,'g_cluster' + 'K' + str(k)] = geodesic.labels.copy()
        df.loc[:,'m_cluster' + 'K' + str(k)] = manhattan.labels.copy()
        df.loc[:,'s_cluster' + 'K' + str(k)] = street.labels.copy()

        end = datetime.datetime.now()
        print(end-start)

    df = df.drop('Points', axis=1)
    try:
        os.remove('./datamound/street/'+ folder + '-' + file.split('.csv')[0] + '.js')
    except FileNotFoundError:
        pass

    df.to_file('./datamound/street/'+ folder + '-' +file.split('.csv')[0] + '.js', driver='GeoJSON')

    print('./datamound/street/'+ folder + '-' +file.split('.csv')[0] + '.js')


In [107]:
year = 2016
folder = 'data_' + str(year)
file = 'm_theft_jan.csv'
krange = range(2,7)
df = pd.read_csv('../data/' + folder +'/' + file, sep =';')

minLat = 42.982366
maxLat = 43.083801
maxLon = -87.855113
minLon = -88.027548

x = []
y = []

df['Points'] = df['Location'].apply(gen_coords)
df['lat'] = df['Points'].apply(lambda x: x[0]) 
df['lon'] = df['Points'].apply(lambda x: x[1])
#print(df.describe())
df = df[(df.lat >= minLat) & (df.lat <= maxLat) & (df.lon >= minLon) & (df.lon <= maxLon)]
print(len(df))

points = [Point(xy) for xy in zip(x,y)]
points = [p for p in points if (p.y >= minLat) & (p.y <= maxLat) & (p.x >= minLon) & (p.x <= maxLon)]
print(len(points))

crs = {'init': 'epsg:4326'}
geo_df = GeoDataFrame(df,crs=crs, geometry=points)
geo_df.head()

geo_df = geo_df.sample(n=200)
X = np.array(geo_df['Points'].tolist())
#print(X)
dfcopy = geo_df.copy()
KMeansInstance(year, X, krange, file, dfcopy)

389
389
data_2016 m_theft_jan.csv 2
Done Street
Fitted Street
Calculating Bias
Storing Bias
0:04:05.028615
data_2016 m_theft_jan.csv 3
Done Street
Fitted Street
Calculating Bias
Storing Bias
0:12:19.625823
data_2016 m_theft_jan.csv 4
Done Street
Fitted Street
Calculating Bias
Storing Bias
0:29:27.019418
data_2016 m_theft_jan.csv 5
Done Street
Fitted Street
Calculating Bias
Storing Bias
0:15:50.676380
data_2016 m_theft_jan.csv 6
Done Street
Fitted Street
Calculating Bias
Storing Bias
0:25:08.908624
./datamound/street/data_2016-m_theft_jan.js


In [52]:
# DONT RUN ME 
input("ARE YOU SURE??!?!?!?!?")
for year in range(2005, 2017):
    folder = 'data_' + str(year)
    for file in os.listdir('../data/' + folder):
        if(file.endswith('.csv')):
            df = pd.read_csv('../data/' + folder +'/' + file, sep =';')
            #print(folder + " " + file)
            #print(df.head())
            print(len(df))
            #df = df.sample(n=100)
            #print(df.head())
            print(len(df))
            #input("Press Enter to continue...")
            start = datetime.datetime.now()
            x = []
            y = []

            df['Points'] = df['Location'].apply(gen_coords)
            points = [Point(xy) for xy in zip(x,y)]
            crs = {'init': 'epsg:4326'}
            geo_df = GeoDataFrame(df,crs=crs, geometry=points)
            print(geo_df.head())
            theft_both = geo_df.copy()
            #print(theft_both.columns)
            test_list = []

            #for index in df['Unnamed: 0']:
            for index in range(len(theft_both)):
                test_list.append(df.loc[index, 'Points'])

            X = np.array(test_list)
            print(X)
            break
            for k in range(2,5):
                print(folder + " " + file + " " + str(k))
                euclid = KMeans(k = k, distance = 'euclid')
                #print("Done Euclid")
                geodesic = KMeans(k = k, distance = 'haversine')
                #print("Done Geodesic")
                manhattan = KMeans(k = k, distance = 'manhattan')
                #print("Done Manhattan")
                street = KMeans(k = k, distance = 'street')
                print("Done Street")
                
                centers = geodesic._init_centers(X)

                euclid.fit(X, centers = centers)
                #print('Fitted Euclid')
                geodesic.fit(X, centers = centers) 
                #print('Fitted Geodesic')
                manhattan.fit(X, centers = centers)
                #print('Fitted Manhattan')
                street.fit(X, centers = centers)
                print('Fitted Street')

                print('Calculating Bias')
                bias_GeodesicEuclid = bias_index(X, geodesic.labels, euclid.labels, demographics, k)
                
                bias_ManhattanGeodesic = bias_index(X, manhattan.labels, geodesic.labels, demographics, k)
                bias_ManhattanEuclid = bias_index(X, manhattan.labels, euclid.labels, demographics, k)
                
                bias_StreetEuclid = bias_index(X, street.labels, euclid.labels, demographics, k)
                bias_StreetGeodesic = bias_index(X, street.labels, geodesic.labels, demographics, k)
                bias_StreetManhattan = bias_index(X, street.labels, manhattan.labels, demographics, k)
                
                print('Storing Bias')
                #  print(folder,file,bias_val,k)
                month = file.split('.csv')[0].split("_")[-1]
                crime_type = file.split(month)[0]
                store_bias(year, month, crime_type, bias_GeodesicEuclid, k)
                store_bias(year, month, crime_type, bias_ManhattanGeodesic, k)
                store_bias(year, month, crime_type, bias_ManhattanEuclid, k)
                store_bias(year, month, crime_type, bias_StreetEuclid, k)
                store_bias(year, month, crime_type, bias_StreetGeodesic, k)
                store_bias(year, month, crime_type, bias_StreetManhattan, k)
                
                ("Saving")
                theft_both.loc[:,'e_cluster' + 'K' + str(k)] = euclid.labels.copy()
                theft_both.loc[:,'g_cluster' + 'K' + str(k)] = geodesic.labels.copy()
                theft_both.loc[:,'m_cluster' + 'K' + str(k)] = manhattan.labels.copy()
                theft_both.loc[:,'s_cluster' + 'K' + str(k)] = street.labels.copy()

                end = datetime.datetime.now()
                print(end-start)
                #  print(percent_similarity(euclid.labels, geodesic.labels))
#                if k == 2:
#                    try:
#                        os.remove('./datamound/'+ folder + '-' + file.split('.csv')[0] + '.js')
#                    except FileNotFoundError:
#                        pass
#                    theft_both.to_file('./datamound/'+ folder + '-' +file.split('.csv')[0] + '.js', driver='GeoJSON')
#                else:
#                    theft_both.to_file('./datamound/'+ folder + '-' +file.split('.csv')[0] + '.js', mode ='a', driver='GeoJSON')
                
            theft_both = theft_both.drop('Points', axis=1)

            try:
                os.remove('./datamound/'+ folder + '-' + file.split('.csv')[0] + '_bu.js')
            except FileNotFoundError:
                pass

            theft_both.to_file('./datamound/'+ folder + '-' +file.split('.csv')[0] + '_bu.js', driver='GeoJSON')
    #         with open('./datamound/'+file.split('.csv')[0] + '.js', 'r') as original: data = original.read()
    #         with open('./datamound/'+file.split('.csv')[0] + '.js', 'w') as modified: modified.write('var both =' 
    #                                                         + data +';')
            print('./datamound/'+ folder + '-' +file.split('.csv')[0] + '.js')
          #  print('-------')
            break
        


494
494
   Unnamed: 0  Incident Number        Date      Time  Police District  \
0           0         51190073  04/29/2005  10:03 AM              5.0   
1           1         51180032  04/28/2005  06:37 AM              7.0   
2           2         51180180  04/28/2005  06:36 PM              7.0   
3           3         51170114  04/27/2005  01:47 PM              4.0   
4           4         51170124  04/27/2005  02:22 PM              4.0   

             Offense 1 Offense 2  Offense 3  Offense 4  Offense 5  \
0  MOTOR VEHICLE THEFT       NaN        NaN        NaN        NaN   
1  MOTOR VEHICLE THEFT       NaN        NaN        NaN        NaN   
2  MOTOR VEHICLE THEFT       NaN        NaN        NaN        NaN   
3  MOTOR VEHICLE THEFT       NaN        NaN        NaN        NaN   
4  MOTOR VEHICLE THEFT       NaN        NaN        NaN        NaN   

                                   Location               Address  \
0  [43.090407754371284, -87.92952162479058]        4039 N 15TH ST   


[[ 43.10969422 -87.97132239]
 [ 43.09084048 -87.9376739 ]
 [ 43.10745216 -87.93520636]
 [ 43.11089402 -87.95649338]
 [ 43.09163925 -87.94341393]
 [ 43.09566191 -87.93050641]
 [ 43.10737899 -87.95169976]
 [ 43.10609453 -87.93530906]
 [ 43.14485644 -87.95583959]
 [ 43.12439273 -87.94615929]
 [ 43.09550751 -87.94677647]
 [ 43.10775844 -87.96024966]
 [ 43.09409249 -87.93521095]
 [ 43.09523095 -87.9269216 ]
 [ 43.09787003 -87.93704235]
 [ 43.1093875  -87.97257534]
 [ 43.11194652 -87.96592333]
 [ 43.11029661 -87.97263965]
 [ 43.10653665 -87.96156125]
 [ 43.11187748 -87.96363142]
 [ 43.09206784 -87.94003934]
 [ 43.10478695 -87.97559957]
 [ 43.11634425 -87.96110386]
 [ 43.09697774 -87.93493214]
 [ 43.12727236 -87.96363151]
 [ 43.09429982 -87.94268668]
 [ 43.10600422 -87.95283386]
 [ 43.14471337 -87.96571863]
 [ 43.09532047 -87.92807641]
 [ 43.13035443 -87.9624552 ]
 [ 43.11005484 -87.95283065]
 [ 43.12526592 -87.966103  ]
 [ 43.10452745 -87.9502725 ]
 [ 43.10989242 -87.9702206 ]
 [ 43.10473151

 [ 43.16177118 -88.03143659]]
541
541
   Unnamed: 0  Incident Number        Date      Time  Police District  \
0           0        161210075  04/30/2016  03:16 PM              7.0   
1           1        161200100  04/29/2016  03:28 PM              5.0   
2           2        161180020  04/27/2016  04:11 AM              7.0   
3           3        161180157  04/27/2016  09:44 PM              5.0   
4           4        161180165  04/27/2016  10:35 PM              5.0   

             Offense 1 Offense 2  Offense 3  Offense 4  Offense 5  \
0  MOTOR VEHICLE THEFT       NaN        NaN        NaN        NaN   
1  MOTOR VEHICLE THEFT       NaN        NaN        NaN        NaN   
2  MOTOR VEHICLE THEFT       NaN        NaN        NaN        NaN   
3  MOTOR VEHICLE THEFT       NaN        NaN        NaN        NaN   
4  MOTOR VEHICLE THEFT       NaN        NaN        NaN        NaN   

                                   Location            Address  \
0  [43.106004220093496, -87.95283385688754

In [67]:
bias_frame.head()

,year,k,theft_jan,theft_feb,theft_mar,theft_apr,theft_may,theft_jun,theft_jul,theft_aug,...,robbery_apr,robbery_may,robbery_jun,robbery_jul,robbery_aug,robbery_sep,robbery_oct,robbery_nov,robbery_dec,m_theft__jan
0,2005,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2005,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2005,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2005,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
file = 'm_theft_jan.js'
data = 'var dataA = ['
year = 2016
year_string = 'data_' + str(year) + '-'

reader = open('./datamound/street/'+ year_string + file,'r')
data += (reader.read() + ',')
reader.close()

writer = open('StreetA.js','w')
writer.write(data + '];')
writer.close()

In [87]:
import os
ordered_names = ['m_theft_jan.js','m_theft_feb.js','m_theft_mar.js','m_theft_april.js',
            'm_theft_may.js','m_theft_june.js','m_theft_july.js','m_theft_aug.js',
            'm_theft_sep.js','m_theft_oct.js','m_theft_nov.js','m_theft_dec.js']

data = 'var dataB = ['
for year in range(2011,2017):

    year_string = 'data_'+str(year)+'-'

    for file in ordered_names:
        reader = open('./datamound/street/'+ year_string + file,'r')
        data += (reader.read() + ',')
        reader.close()
        print(file)
        
    writer = open('halfB.js','w')
    writer.write(data + '];')
    writer.close()

    
data = 'var dataA = ['
for year in range(2005,2011):

    year_string = 'data_'+str(year)+'-'

    for file in ordered_names:
        reader = open('./datamound/'+ year_string + file,'r')
        data += (reader.read() + ',')
        reader.close()
        print(file)

    writer = open('halfA.js','w')
    writer.write(data + '];')
    writer.close()


theft_jan.js
theft_feb.js
theft_mar.js
theft_april.js
theft_may.js
theft_june.js
theft_july.js
theft_aug.js
theft_sep.js
theft_oct.js
theft_nov.js
theft_dec.js
m_theft_jan.js
m_theft_feb.js
m_theft_mar.js
m_theft_april.js
m_theft_may.js
m_theft_june.js
m_theft_july.js
m_theft_aug.js
m_theft_sep.js
m_theft_oct.js
m_theft_nov.js
m_theft_dec.js
theft_jan.js
theft_feb.js
theft_mar.js
theft_april.js
theft_may.js
theft_june.js
theft_july.js
theft_aug.js
theft_sep.js
theft_oct.js
theft_nov.js
theft_dec.js
m_theft_jan.js
m_theft_feb.js
m_theft_mar.js
m_theft_april.js
m_theft_may.js
m_theft_june.js
m_theft_july.js
m_theft_aug.js
m_theft_sep.js
m_theft_oct.js
m_theft_nov.js
m_theft_dec.js
theft_jan.js
theft_feb.js
theft_mar.js
theft_april.js
theft_may.js
theft_june.js
theft_july.js
theft_aug.js
theft_sep.js
theft_oct.js
theft_nov.js
theft_dec.js
m_theft_jan.js
m_theft_feb.js
m_theft_mar.js
m_theft_april.js
m_theft_may.js
m_theft_june.js
m_theft_july.js
m_theft_aug.js
m_theft_sep.js
m_theft_oct.j

In [11]:
bias_frame.to_json('bias.js',orient='records')
with open('bias.js','r') as reader:
    data = reader.read()
with open('bias.js','w') as w:
    w.write('var bias_data =' + data + ';')
    